In [77]:
#import packages for analysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#set paths to covid data
state_path = 'covid_19_data/us-states.csv'
nation_path = 'covid_19_data/us.csv'

#set path to unemployment data
unemp_path = 'unemployment_data/Unemployment_Data_adj.csv'

#import data to pandas dataframes
cov_state_df = pd.read_csv(state_path)
cov_nat_df = pd.read_csv(nation_path)
unemp_df = pd.read_csv(unemp_path)

for df in [cov_state_df, cov_nat_df, unemp_df]:
    print(df.info(), '\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12664 entries, 0 to 12663
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    12664 non-null  object
 1   state   12664 non-null  object
 2   fips    12664 non-null  int64 
 3   cases   12664 non-null  int64 
 4   deaths  12664 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 494.8+ KB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    272 non-null    object
 1   cases   272 non-null    int64 
 2   deaths  272 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.5+ KB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 11 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  

In [78]:
#Join cov_state and cov_nat dfs on the 'date' index
cov_tot_df = cov_state_df.set_index('date').join(cov_nat_df.set_index('date'), lsuffix='_state', rsuffix='_national')
#cov_tot_df.to_csv('combined_data.csv', index=True)

#Summarize table
cov_tot_df.info()

#Result is a pandas dataframe where each row of state data and cases is accompanied by the national data at that time
cov_tot_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12664 entries, 2020-01-21 to 2020-10-18
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   state            12664 non-null  object
 1   fips             12664 non-null  int64 
 2   cases_state      12664 non-null  int64 
 3   deaths_state     12664 non-null  int64 
 4   cases_national   12664 non-null  int64 
 5   deaths_national  12664 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 692.6+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 12664 entries, 2020-01-21 to 2020-10-18
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   state            12664 non-null  object
 1   fips             12664 non-null  int64 
 2   cases_state      12664 non-null  int64 
 3   deaths_state     12664 non-null  int64 
 4   cases_national   12664 non-null  int64 
 5   deaths_national  12664 non-null  i

In [79]:
#Next we want to join the relevant unemployment data with our cov_tot_df such that each tuple in the df consists of a
#state name, state level cases and deaths of covid_19, national cases and deaths of covid_19, and the unemployment rate
#of the state for that month. We can do this by slicing the date string

cov_tot_df = cov_tot_df.reset_index()

#create list of month integers from 'date' column
month_data = []
for date in cov_tot_df['date']:
    month_data.append(int(date[5:7]))

#store as a pandas series and append to cov_tot_df
month_series = pd.Series(month_data)
cov_tot_df['month'] = month_series

#perform join on unemp_df
unemp_df = unemp_df.rename(columns={'State and area': 'state', 'Month': 'month'})
cov_unemp_df = pd.merge(cov_tot_df, unemp_df, how='left', on=['month', 'state'])

#drop data from after august (no unemployment data) AND from US territories
territories = ['Puerto Rico', 'Virgin Islands', 'Northern Mariana Islands', 'Guam']

temp = cov_unemp_df.drop(cov_unemp_df[cov_unemp_df['month'] > 8].index)
clean_data = temp.drop(temp[temp['state'].isin(territories)].index)

#this gives us a clean dataset with no null values, containing only information from 50 states.
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9354 entries, 0 to 10023
Data columns (total 17 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   date                                   9354 non-null   object 
 1   state                                  9354 non-null   object 
 2   fips                                   9354 non-null   int64  
 3   cases_state                            9354 non-null   int64  
 4   deaths_state                           9354 non-null   int64  
 5   cases_national                         9354 non-null   int64  
 6   deaths_national                        9354 non-null   int64  
 7   month                                  9354 non-null   int64  
 8   FIPS Code                              9354 non-null   float64
 9   Year                                   9354 non-null   float64
 10  Civilian non-institutional population  9354 non-null   object 
 11  Lab

In [80]:
#Write cleaned data to a .csv for further analysis
clean_data.to_csv('cleaned_covid19_unemployment_data.csv', index=False)